# IBESI

**IBESI** is an all-encompassing money management system for Oscar and Rose. It stands for:
- **I**ncome
- **B**ills
- **E**xpenses
- **S**avings
- **I**nvestments

These are the key financial areas this system supports.

## Financial Areas
### Income
Income refers to any source of money coming to us. The most obvious instance of income is our salaries.

### Bills
Bills are the payments performed in cycles. These include mortgage, utility bills and similar.

### Expenses
Expenses are any ad-hoc payments for goods or services. These include food, cleaning and shopping.

### Savings
This is the share of money left after using our income to pay for bills and expenses that is saved.

### Investments
This is the share of money left after savings that is put back in the market to generate a return.

## Money Pipeline
The money processing pipeline is as shown below.

<!-- ![pipeline](ibesi-pipeline.png) -->


## Components
The components are the steps of the money pipeline. These operations are part of the core logic of the system and should not be modified without a system design review.

## Input Data
- Income amount
- Savings accumulated
- Bills for the past month
- Expenses for the past month
- Investments in the past month

## Parameters
Parameters are the parts of the system we can modify to make it work for our financial situation and expectations. These include:
- Saving pots and shares for each of them
- How bills and expenses are split between us
- Amount of pocket money for personal expenses

In [4]:
from data.D2312 import INPUTS, PARAMS


def inspect_inputs_dict():
    pass


def inspect_params_dict():
    pass


def get_income(inputs):
    income_oscar = sum(source["oscar"] for source in inputs["income"].values())
    income_rose = sum(source["rose"] for source in inputs["income"].values())
    return income_oscar, income_rose

def get_bills(inputs):
    bills_oscar = sum(source["oscar"] for source in inputs["bills"].values())
    bills_rose = sum(source["rose"] for source in inputs["bills"].values())
    return bills_oscar, bills_rose


def get_expenses(inputs):
    expenses_oscar = sum(source["oscar"] for source in inputs["expenses"].values())
    expenses_rose = sum(source["rose"] for source in inputs["expenses"].values())
    return expenses_oscar, expenses_rose


def get_investments(inputs):
    investments_oscar = sum(
        source["oscar"] for source in inputs["investments"].values()
    )
    investments_rose = sum(source["rose"] for source in inputs["investments"].values())
    return investments_oscar, investments_rose


def get_spending_total(inputs):
    return sum(get_bills(inputs)) + sum(get_expenses(inputs))


def get_spending_share(inputs, params):
    if params["split"] == "proportional_to_income":
        income_oscar, income_rose = get_income(inputs)
        split_percentage_oscar = income_oscar / (income_oscar + income_rose)
        split_percentage_rose = income_rose / (income_oscar + income_rose)
        return split_percentage_oscar, split_percentage_rose
    else:
        raise ValueError("split mode not supported")


def get_expected_spending(inputs, params):
    spending_share_oscar, spending_share_rose = get_spending_share(inputs, params)
    expected_spending_oscar = get_spending_total(inputs) * spending_share_oscar
    expected_spending_rose = get_spending_total(inputs) * spending_share_rose
    return expected_spending_oscar, expected_spending_rose


def get_actual_spending(inputs):
    actual_spending_oscar = (
        sum(source["oscar"] for source in inputs["bills"].values())
        + sum(source["oscar"] for source in inputs["expenses"].values())
        + sum(source["oscar"] for source in inputs["investments"].values())
    )
    actual_spending_rose = (
        sum(source["rose"] for source in inputs["bills"].values())
        + sum(source["rose"] for source in inputs["expenses"].values())
        + sum(source["rose"] for source in inputs["investments"].values())
    )
    return actual_spending_oscar, actual_spending_rose


def get_split_difference(inputs, params):
    expected_spending_oscar, expected_spending_rose = get_expected_spending(
        inputs, params
    )
    actual_spending_oscar, actual_spending_rose = get_actual_spending(inputs)
    split_difference_oscar = expected_spending_oscar - actual_spending_oscar
    split_difference_rose = (expected_spending_rose - actual_spending_rose,)
    return split_difference_oscar, split_difference_rose


def get_surplus(inputs):
    # TODO: find a better name
    income_oscar, income_rose = get_income(inputs)
    actual_spending_oscar, actual_spending_rose = get_actual_spending(inputs)
    surplus_oscar = income_oscar - actual_spending_oscar
    surplus_rose = income_rose - actual_spending_rose
    return surplus_oscar, surplus_rose


def get_pocket_money(params):
    pocket_money_oscar = params["pocket_money"]["oscar"]
    pocket_money_rose = params["pocket_money"]["rose"]
    return pocket_money_oscar, pocket_money_rose


def get_savings(inputs, params):
    surplus_oscar, surplus_rose = get_surplus(inputs)
    pocket_money_oscar, pocket_money_rose = get_pocket_money(params)
    savings_oscar = surplus_oscar - pocket_money_oscar
    savings_rose = surplus_rose - pocket_money_rose
    return savings_oscar, savings_rose


def get_saving_pots_contributions(inputs, params):
    savings_total = sum(get_savings(inputs, params))
    saving_pots_shares = {}
    for pot in params["saving_pots_shares"]:
        saving_pots_shares[pot] = savings_total * params["saving_pots_shares"][pot]
    return saving_pots_shares


print(get_spending_share(INPUTS, PARAMS))
print(get_spending_total(INPUTS))
print(get_actual_spending(INPUTS))
print(get_split_difference(INPUTS, PARAMS))
print(get_surplus(INPUTS))
print(get_savings(INPUTS, PARAMS))
print(get_saving_pots_contributions(INPUTS, PARAMS))

(0.6584251545283526, 0.3415748454716474)
3357.4800000000005
(2566.73, 1040.75)
(-356.08071217414636, (106.08071217414681,))
(2333.27, 1501.25)
(1833.27, 1001.25)
{'new_house': 850.356, 'car': 566.904, 'current_house': 425.178, 'holiday': 425.178, 'wedding': 283.452, 'children': 141.726, 'family': 141.726}
